In [ ]:
import pandas as pd

#For text processing
#Using this (https://bit.ly/2HvV2dx) as template
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer #, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
tokenizer = RegexpTokenizer(r'\w+')
import numpy as np
np.random.seed(2018)
import nltk


In [ ]:
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
# import pandas as pd

df=pd.read_pickle('dftst.pkl')
print (len(df))
# df['review'][0]
df.head(50)

df_work=df.copy()
df_work.head(-10)

In [ ]:
# reviews_per_trail=df_work.groupby('name').count()
# # reviews_per_trail['review'].hist()
# # plt.show()
# print(reviews_per_trail)

In [ ]:
# # Data pre-processing - text cleaning

# df_r=df['review']


# def preprocess(datframe):
#     datframe.apply(lambda x: x.lower())
#     datframe.apply(lambda x: x.translate(None, string.punctuation))
#     datframe.apply(lambda x: x.translate(None,string.digits))
#     return (datframe)

# # for i in df_r:
# #     print(preprocess(str(i)))
    
# preprocess(df['review'])
# # df_r.head()

# # tokens=tokenizer.tokenize(df_r)

# # def data_preprocess(dat):
# #     tokens=tokenizer.tokenize(df_r)
# # #     words=
# #     lemmatize=stemmer.stem(WordNetLemmatizer().lemmatize(dat, pos='v'))
# #     tokenizer = RegexpTokenizer(r'\w+')
                           


In [ ]:
# def lemmatize_stemming(text):
#     return PorterStemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    words=[word for sentence in sent_tokenize(text) for word in word_tokenize(sentence)]
    words=[word for word in words if len(word)>=3]
    words=[word for word in words if word not in gensim.parsing.preprocessing.STOPWORDS]
    words=[WordNetLemmatizer().lemmatize(word) for word in words]
    #Lemmatize
#     words=[WordNetLemmatizer.lemmatize(word) for word in words]
    return words

In [ ]:
import string
import re
reviews=df_work.review
#tokenize text
# reviews.apply(lambda x: x.lower())
reviews=reviews.str.lower()
reviews=reviews.str.replace('\"', '')
reviews=reviews.str.replace("\'", '')
reviews=reviews.str.replace('\"', '')
reviews=reviews.str.replace('\!', '')
reviews=reviews.str.replace('\/', ' ')
reviews=reviews.str.replace(',', '')
reviews=reviews.str.replace('(', '')
reviews=reviews.str.replace(')', '')
reviews=reviews.str.replace('.', '')
reviews=reviews.str.replace('\d+', '')
reviews=reviews.replace(to_replace={'hik', 'walk','ing','run','trail', 'interesting',\
                                    'good','great','lot','recommend','area','park',\
                                    'love','like','way','easy'}, value='', regex=True)

processed_text=[preprocess(text) for text in reviews]
# lemmatized_text=[WordNetLemmatizer.lemmatize(text) for text in reviews]
# WordNetLemmatizer.lemmatize('are')

# reviews.head(50)
print (processed_text[0], reviews[0])
# print ()

In [ ]:
#Add processed text as a new column
df_work['processed_text']= pd.Series(processed_text)
df_work.head()
#[' '.join(filter(None,filter(lambda word: word not in stop, line))) for line in data['text_data'].str.lower().str.split(' ')]

In [ ]:
df_work.processed_text[0]

In [ ]:
#Creating word countf for IDF
from sklearn.feature_extraction.text import CountVectorizer
import re
 
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)
 
#load a set of stop words
stopwords=get_stop_words("./stop_words.txt")
 
#get the text column 
docs=df_work['processed_text'][0] #.tolist()
 
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

In [ ]:
list(cv.vocabulary_.keys())[:20]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
 

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(df_work['processed_text'])
corpus = [dictionary.doc2bow(text) for text in df_work['processed_text']]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Topic modeling based on this: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [ ]:
# processed_text
dictionary = gensim.corpora.Dictionary(processed_text)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
len(dictionary)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)
len(dictionary)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]
print (len(bow_corpus))
bow_corpus[41]

In [ ]:
bow_doc_400 = bow_corpus[10]
for i in range(len(bow_doc_400)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_400[i][0], 
                                               dictionary[bow_doc_400[i][0]], 
bow_doc_400[i][1]))

In [ ]:
#Bag of words
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
#Train lda model using gensim.models.LdaMulticore and save it to ‘lda_model’
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
#Explore words in topic and relative weight
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
#LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
#Performance evaluation-LDA bag of words
print (processed_text[40])
for index, score in sorted(lda_model[bow_corpus[40]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
#Performance evaluation-LDA TF-IDF
print (processed_text[40])
for index, score in sorted(lda_model_tfidf[bow_corpus[40]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
# # run word2vec model and then save it
# texts_stemmed = filter(None, [next_text.strip(' ').split(' ') for next_text in df_work['processed_text'][0]])
# w2vmodel_stemmed = gensim.models.Word2Vec(texts_stemmed, size=100, window=5, min_count=5, workers=4)
# w2vmodel_stemmed.save(savefolder+'w2v_stemmed_model')